<a href="https://colab.research.google.com/github/Apofice2/RNN_Training.ipynb/blob/main/Segmentation_de_documents_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapitre 1 : Prétraitement de fichiers texte

## 1. Segmentation et Tokenization

### 1.1 From scratch ou presque

On commence par la méthode la plus directe de segmentation. Elle s'appuie sur la méthode standard $\texttt{str.split()}$ de la classe $\texttt{str}$

->https://docs.python.org/fr/3.6/library/stdtypes.html?highlight=split#str.split

In [ ]:
sentence='''Plato was a famous greek philosopher whose dialogue Cratylus mainly deals with the correctness of names. \n '''
print(sentence)
sentence.split()

C'est un résultat satisfaisant en première approximation. A noter cependant 'philosopher.' et 'names.' Certains jetons associent segmentation et marques de ponctuation. Ce genre de "détails" rend délicate la conception de Tokenizer versatiles.

### 1.2 Expressions régulières et segmentation

Dans l'exemple précédent, on peut améliorer les choses en travaillant avec des $\textbf{expressions régulières}$. Il s'agit alors de se donner des règles générales d'analyse permettant d'isoler ou même effacer certains caractères dans des environnements précis.

-> https://docs.python.org/fr/3/library/re.html

In [ ]:
import re

tokens=re.split(r'[-\s.,;!?]',sentence)
print(tokens)
#pour enlever les symboles . et ''
tokens_cleared=[x for x in tokens if x and x not in '- \t\n.,;!?']
print(tokens_cleared)

### 1.3 Tokenizers de référence

Un grand nombre de librairies python centrées sur le traitement du langage naturel contiennent des implémentations de Tokenizer beaucoup plus sophistiqués que les méthodes décrites ci-dessus.

-> https://spacy.io

-> https://stanfordnlp.github.io/CoreNLP/, https://corenlp.run

-> https://www.nltk.org

On peut par exemple utiliser le Treebank Word Tokenizer de la dernière librairie pour segmenter notre phrase de travail

In [ ]:
from nltk.tokenize import TreebankWordTokenizer,punkt
tokenizer=TreebankWordTokenizer()
tokens=tokenizer.tokenize(sentence)
print(tokens)

## 2. Normalisation du vocabulaire

Par normalisation du vocabulaire, on entend la gestion des modifications suivantes : prise en compte des majuscules, identification des racines apparentées, lemmatisation. La prise en compte de ces aspects permet d'affiner la représentation des textes et documents.

### 2.1 Gestion des majuscules

C'est le degré zéro de la normalisation. Dans la plupart des cas, il n'est pas nécessaire de distinguer les mots 'Phénomène' et 'phénomène' qui ne diffèrent que par la présence d'une majuscule.  On oublie facilement ces différences via la méthode $\texttt{lower}$ de la classe $\texttt{str}$.

In [ ]:
print(tokens)
tokens=[x.lower() for x in tokens]
print(tokens)

### 2.2 Prise en compte des "stop words"

Les $\textbf{stop words}$ sont les mots outils/liaisons d'une langue dont on peut en général considérer qu'ils véhiculent peu d'information sémantique. Il s'agit des articles, de certaines particules, de certaines interjections. On est donc tenté de les faire disparaître de la liste des jetons caractéristiques de notre phrase / document.

In [ ]:
stop_words = ['the','and','a','an','this','that','of']
#tokens=tokenizer.tokenize(sentence)
print(tokens)
tokens = [x for x in tokens if x not in stop_words]
print(tokens)


Mais on peut utiliser des listes pré-construites associées aux librairies mentionnées.

Voici la liste fournie par NLTK

In [ ]:
import nltk

nltk.download('stopwords')
stop_words=nltk.corpus.stopwords.words('german')

print(len(stop_words))
stop_words[:10]

Satz=''' Kennst du das Land, wo die Zitronen bluhn '''
Zeichen=tokenizer.tokenize(Satz)

Zeichen = [x for x in Zeichen if x not in stop_words]
print(Zeichen)

Le module $\texttt{sklearn.feature_extraction}$ de Scikitlearn possède également sa propre liste de stop words.  

-> https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

Toutes ces listes ne sont pas totalement identiques et sont mises à jour régulièrement.

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words
len(sklearn_stop_words)

Attention ! Dans certains cas, les majuscules sont porteuses de sens !

### 2.3 Racinisation ou "stemming"

Il s'agit de ne pas prendre en compte les variations d'écriture induites par : les marques du pluriel, les marques possessives et les déclinaisons. C'est une tâche bien plus délicate qu'il n'y paraît. Elle a pour objectif l'identification de jetons du vocabulaire à partir de critère morphologique.

On peut bien sûr procéder de manière directe. Voici par exemple une fonction qui élimine tous les 's' en position finale des mots. Cette racinisation manuelle repose entièrement sur la manipulation d'expression régulière.

In [ ]:
import re

def stem_basic(phrase):
    return ' '.join([re.findall('^(.*ss|.*?)(s)?$',word)[0][0].strip("'") for word in phrase.lower().split()])

word_test=stem_basic("nombres")

print(word_test)

phrase_test ="Les heures pourpres des nombres gris"
phrase_test_stemmed = stem_basic(phrase_test)

print(phrase_test_stemmed)


Il existe bien entendu des stemmers très sophistiqués $\textbf{adaptés aux spécificités de chaque langue}$.

Pour l'anglais, les algorithmes de racinisation les plus utilisés sont Snowball et Porter.

Voir https://tartarus.org/martin/PorterStemmer/, pour le dernier.

Voir également https://github.com/jedijulia/porter-stemmer/blob/master/stemmer.py pour une implémentation 100% python de l'algorithme original.

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer

ps=PorterStemmer()
snow=SnowballStemmer(language='english')

stop_words=nltk.corpus.stopwords.words('english')

print(tokens)
tokens_ps = [ps.stem(x) for x in tokens if x not in stop_words]
tokens_bis = [snow.stem(x) for x in tokens if x not in stop_words]
print(tokens_ps)
print(tokens_bis)


### 2.4 Lemmatisation

La lemmatisation permet, comme la racinisation, d'identifier certains jetons. A la différence de cette dernière, elle s'appuie sur des informations d'ordre $\textbf{sémantique}$.

Elle s'aide également d'informations grammaticales telles que la $\textbf{POS (Part of Speech)}$; qui correspond par exemple au deuxième argument de la fonction $\texttt{lemmatize}$ implémentée dans NLTK.

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

print(lemmatizer.lemmatize("manned", pos="v"))

print(lemmatizer.lemmatize("manned", pos="a"))

print(lemmatizer.lemmatize("calls", pos="v"))

print(lemmatizer.lemmatize("called", pos="v"))

Si l'on dispose donc d'un étiquetage de la phrase (document) en terme de POS, on peut appliquer le lemmatizer à chaque jeton en tenant compte de cette information supplémentaire.

Pour plus d'information, on pourra par exemple consulter :

-> https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

## 3. D'autres jetons : les n-grams

Jusqu'à présent (et ce sera globalement le cas dans ce notebook), les jetons ont été implicitement pensés comme des mots. Mais les mots ne sont pas les seules entités porteuses de sens. Il arrive que des suites de plusieurs mots (ou jetons après tokenization) correspondent à une unité sémantique. Il est alors logique d'associer un seul jeton à une suite de plusieurs mots.

Dans le cadre de la segmentation, ces suites de $n$ mots sont appelées $\textbf{n-grams}$. La librairie NLTK dispose de fonctions permettant de les intégrer à la segmentation. Un exemple ci-dessous.

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def tokenize_ngrams(text, n ):
    n_grams = ngrams(word_tokenize(text), n)
    return [ ' '.join(grams) for grams in n_grams]

tokens_2grams=tokenize_ngrams(sentence,2)
tokens_3grams=tokenize_ngrams(sentence,3)
print(tokens_2grams)
print(tokens_3grams)

## 4. Représentations vectorielles (première partie)

### 4.1 Représentation vectorielle "one-hot-vector"

On va s'appuyer sur la segmentation précédente afin de créer une représentation vectorielle élémentaire de notre phrase de référence. Une représentation "one-hot-vector" repose sur la donnée du vocabulaire à partir duquel a été composé le document.

A chaque jeton du document, on associe un vecteur qui contient uniquement des zéros, à l'exception de la coordonnée du mot du vocabulaire auquel il correspond.

In [ ]:
import numpy as np

voc = sorted(set(tokens))
print(voc)
','.join(voc)
n_tokens = len(tokens)
n_voc = len(voc)

OHV = np.zeros((n_tokens,n_voc),int)

for i,word in enumerate(tokens):
    OHV[i,voc.index(word)]=1

#' '.join(voc)
print(OHV)

Afin de rendre cette représentation plus lisible, on peut utiliser pandas. Ceci permet notamment d'étiqueter les colonnes.

In [ ]:
import pandas as pd

df=pd.DataFrame(OHV, columns=voc)
df[df==0]=''
df

Il s'agit d'une première manière d'associer une représentation vectorielle "numérique" à une chaine de caractère. Cette méthode est simple mais gourmande en terme de mémoire.

A noter : on rencontrera à nouveau ce type de représentation dans le cadre de la construction des word embedding (Word2vec en particuluer)

### 4.2 Représentation vectorielle d'un corpus : Bag-of-words

Supposons à présent que l'on travaille non plus avec un document mais avec $\textbf{un corpus de documents}$ (c'est à dire un ensemble de documents). Le code ci-dessous propose une réprésentation spécifique du corpus.

In [ ]:
sentences=sentence
sentences+='''Socrates was also a famous philosopher who taught Plato in Athens. \n '''
sentences+='''Socrates compares the creation of words to the work of an artist. \n '''
sentences+='''The creation of words uses letters containing certain sounds to express the essence of words subject. \n '''
sentences+='''The famous Hermogenes opposing Socrates suggests that words do not express the essence of their subject'''

print(sentences)
corpus={}
for i,sent in enumerate(sentences.split('\n')):
    #Tokenization à l'échelle de la phrase avec racinisation et nettoyage des stop words
    tokens=tokenizer.tokenize(sent)
    tokens=[ps.stem(x) for x in tokens if x not in stop_words]
    corpus['sent{}'.format(i)]=dict((tok,1) for tok in tokens)

print(corpus)

# Création d'un dataframe à partir du corpus (liste de vecteurs)
df=pd.DataFrame.from_records(corpus).fillna(0).astype(int).T

#df[df.columns[20:]]
df.head(5)

### 4.3 Mesure de similarité

A partir d'une telle représentation vectorielle, il devient possible de comparer les phrases (resp. textes) d'un texte (resp. corpus) entre elles. On utilise à cet effet le produit scalaire aux vecteurs BOW associés aux différentes phrases du texte.

$$ \langle senti, sentj \rangle = \sum_{k=1}^{n_{\mathrm{voc}}} senti[k] sentj[k]$$

In [ ]:
## Statistiques sur df via pandas

tab=df.T
print(df.describe())
print(df.cov())


## Calculs de similarité

print(tab.sent0.dot(tab.sent1))
print(tab.sent1.dot(tab.sent2))
print(tab.sent0.dot(tab.sent3))

# 5. Analyse de sentiment

L'analyse de sentiment est l'une des tâches les plus répandues en NLP. Si l'on peut s'y atteler à l'aide d'outils sophistiqués (réseaux de neurones et autres), souvent nécessaires quand la masse et la complexité des données augmentent. Pour des corpus restreints et des données particulières, on peut parvenir à de bons résultats via des moyens relativement "simples".

## 5.1 Un premier exemple avec VADER

L'algorithme VADER est implémenté dans NLTK mais également dans le package vaderSentiment maintenu par l'auteur de l'algorithme. Il est particulièrement adapté à l'analyse de sentiments pour des textes courts issus des réseaux sociaux ou SMS.



In [ ]:
import sys
!{sys.executable} -m pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sa=SentimentIntensityAnalyzer()
sa.lexicon

L'algorithme associe des scores aux différents messages selon trois items : positivity, negativity et neutral. Il calcule ensuite un score composé à partir de ces trois champs.

In [ ]:
sa.polarity_scores(text="Aristotle is so amazing. His books are just perfect ;-)")

In [ ]:
corpus=["Awesome ! You are the best", "So dumb ! You are useless", "It was so so, well written for sure but boring as well"]

for text in corpus:
    scores=sa.polarity_scores(text)
    print('{:+}: {}'.format(scores['compound'],text))


## 5.2 Classification bayésienne naïve

On commence par récupérer une base de données qui contient des critiques de films auxquelles sont associées des notes. (tirée de Hutto et Gilbert)

In [ ]:
import sys
!{sys.executable} -m pip install nlpia

In [ ]:
from nlpia.data.loaders import get_data
movies=get_data('hutto_movies')

In [ ]:
type(movies)

Un coup d'oeil rapide au data set.

In [ ]:
movies.head().round(3)

In [ ]:
movies.describe().round(2)

### Segmentation et création d'un BOW à partir de movies

On va à présent appliquer le tokenizer et créer un vecteur BOW pour chaque critique de films, exactement comme dans le cadre de la section 4.2. Le tout est emballé dans un DataFrame pandas pour plus de lisibilité.

In [ ]:
import pandas as pd
pd.set_option('display.width',75)

from nltk.tokenize import casual_tokenize # tokenizer adapté à ce type de contenu
from collections import Counter

BOWS=[]
for text in movies.text:
    BOWS.append(Counter(casual_tokenize(text)))

#print(BOWS)
df_BOWS=pd.DataFrame.from_records(BOWS) ## la fonction from_records se charge d'identifier les termes
df_BOWS.head()

df_BOWS=df_BOWS.fillna(0).astype(int)
print(df_BOWS.shape)

df_BOWS.head()
#df_BOWS.head()[list(BOWS[0].keys)]


In [ ]:
df_BOWS.head()[list(BOWS[6].keys())]

On va maintenant appliquer le $\textbf{classificateur bayésien naïf}$ de scikit-learn afin de résoudre le problème de classification.

-> https://scikit-learn.org/stable/modules/naive_bayes.html

In [ ]:
print(movies)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

## Préparation de l'entrainement
X_train, X_test, y_train, y_test = train_test_split(df_BOWS, movies.sentiment > 0, test_size=0.2, random_state=42)

print(y_train)
## Création du classificateur
NBC=MultinomialNB()
#NBC=NBC.fit(X_train, movies.sentiment > 0) # La classe doit avoir un type int ou bool
NBC=NBC.fit(X_train, y_train)




## Homogeneisation des notes vs proba
pred_train_cat=NBC.predict_proba(X_train)
print("pred_train_cat",pred_train_cat)
test_train_cat=pred_train_cat[:,0]<pred_train_cat[:,1]
error_train_cat=abs(test_train_cat.T!=y_train).mean()
print("error_train",error_train_cat)
#error.mean()

pred_test_cat=NBC.predict_proba(X_test)
print("pred_train_cat",pred_test_cat)
test_test_cat=pred_test_cat[:,0]<pred_test_cat[:,1]
error_test_cat=abs(test_test_cat.T!=y_test).mean()
print("error_test",error_test_cat)
#error.mean()


L'efficacité demeure-t-elle lorsqu'on change de données ? On va tester l'approche précédente sur un data set un peu différent qui contient des critiques de produits et non plus de films

In [ ]:
....

Le résultat n'est pas aussi satisfaisant que précédemment.

Pourquoi ? Quelles pistes d'amélioration ?